In [ ]:
from config import OMEROUSER, OMEROPASS, OMEROHOST, OMEROPORT, OMEROWEB
from json_session import create_json_session, retrieve_image
import matplotlib.pyplot as plt
import requests
import json

login_rsp, session, base_url = create_json_session(OMEROWEB, OMEROUSER, OMEROPASS, verify=False)

In [ ]:
r = session.get(base_url)
urls = r.json()
print(json.dumps(urls, indent=2))

In [ ]:
servers_url = urls['url:servers']
servers_json = session.get(servers_url).json()
print(json.dumps(servers_json, indent=2))

In [ ]:
projects_url = urls['url:projects']
user1_url = projects_url+"?owner=553"
user1_json = session.get(user1_url).json()
print(json.dumps(user1_json, indent=1))

In [ ]:
myproj_id = 557
myproj_url = projects_url+str(myproj_id)+"/"
myproj_json = session.get(myproj_url).json()
print(json.dumps(myproj_json, indent=1))

In [ ]:
data_in_proj_url = myproj_url + "datasets/"
print(data_in_proj_url)
data_in_proj_json = session.get(data_in_proj_url).json()
print(json.dumps(data_in_proj_json, indent=1))

In [ ]:
ds_id = 858
data_url = urls['url:datasets'] + str(ds_id) + "/"
print(data_url)
data_json = session.get(data_url).json()
print(json.dumps(data_json, indent=1))

In [ ]:
images_in_ds_url = data_url + "images/"
print(images_in_ds_url)

In [ ]:
image_id = 6013
image_url = urls['url:images'] + str(image_id) + "/"
print(image_url)
image_json = session.get(image_url).json()
print(json.dumps(image_json, indent=1))

In [ ]:
rois_url = image_url + "rois/"
print(rois_url)
rois_json = session.get(rois_url).json()
print(json.dumps(rois_json, indent=1))

In [ ]:
scale_factor=1
img = retrieve_image(session, base_url, image_id, scale_factor)
plt.imshow(img)